## Get started and connect to ES

In [21]:
import boto3
import json
from jsonschema import validate
from elasticsearch import Elasticsearch, RequestsHttpConnection
from elasticsearch.helpers import scan
from requests_aws4auth import AWS4Auth
import pandas as pd
import requests
from tika import parser
import PyPDF2
import os

region = 'us-east-1'
service = 'es'
eshost = 'search-world-modelers-dev-gjvcliqvo44h4dgby7tn3psw74.us-east-1.es.amazonaws.com'

session = boto3.Session(region_name=region, profile_name='wmuser')
credentials = session.get_credentials()
credentials = credentials.get_frozen_credentials()
access_key = credentials.access_key
secret_key = credentials.secret_key
token = credentials.token

aws_auth = AWS4Auth(
    access_key,
    secret_key,
    region,
    service,
    session_token=token
)

es = Elasticsearch(
    hosts = [{'host': eshost, 'port': 443}],
    http_auth=aws_auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

index = 'wm-dev'

### Ingest MITRE Spreadsheet and obtain stats

In [9]:
xls = pd.ExcelFile('/Users/brandon/Downloads/Six_Twelve-Month_November_2019%20Evaluation%20Documents%20-%20Updated%20-%2026%20June%202019.xlsx')

ignore_sheets = ['ReadMe','Introduction','Copyright & Terms of Use']

sheets = {}
for sheet in xls.sheet_names:
    if sheet not in ignore_sheets:
        df = pd.read_excel(xls, sheet)
        sheets[sheet] = df.shape[0]

total = 0
for k in sheets:
    total += sheets[k]
    
print(f"There are {total} total documents in the MITRE spreadsheet. There are the following counts per tab:\n")        
print(json.dumps(sheets, indent=2, sort_keys=True))

There are 1287 total documents in the MITRE spreadsheet. There are the following counts per tab:

{
  "Additional Six-Month Eval Docs": 31,
  "Luma-Provided Ethiopia Docs": 32,
  "November 2019 Ethiopia Docs": 265,
  "November 2019 SSudan Docs": 431,
  "Six-Month Evaluation Documents": 52,
  "Twelve-Month Eval Docs": 476
}


## Garbage text extraction
This document in ES clearly had failed text extraction, however when we open the document there is clearly text. What could be happening? Note: both extractors seem to fail.

If we can't resolve this, we should at least characterize this issue by determining **how many documents are effected by "silently" failed text extraction?** (extractor succeeds but produces too little text).

In [23]:
url_path = 'https://reliefweb.int/sites/reliefweb.int/files/resources/South_Sudan_KeyMessages_Sept2017.pdf'  

query = {
        "query": {
            "match" : {
                "source_url.keyword" : url_path
            }
        }
    }            
            
res = es.search(index=index, body=query)

In [24]:
res

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 1,
  'max_score': 5.1279244,
  'hits': [{'_index': 'wm-dev',
    '_type': 'wm-document',
    '_id': 'a55811aee1d1fa8dbe83fd5ee4045ae69f71b27cb6ec33ae7aa1ae0432181489',
    '_score': 5.1279244,
    '_source': {'stored_url': 'https://world-modelers.s3.amazonaws.com/documents/migration/tmp/DEV/SouthSudanKeyMessagesSept2017.pdf',
     'file_name': 'SouthSudanKeyMessagesSept2017.pdf',
     'file_type': '.pdf',
     'modification_date': {'date': '2017-11-06T05:45:35Z'},
     'creation_date': {'date': '2017-11-06T05:45:35Z'},
     'source': {'author_name': 'Kerandi, Nicholas (FAOSS)'},
     'category': 'Six-Month Evaluation Documents',
     'extracted_text': {'pypdf2': '1\n \n \n\n \n \n\n \n\n \n\n \n5\n \n\n \n\n \n                                        \n                  \n \n3\n \n \n \n2\n \n \n3\n \n \n4\n \n \n5\n \n \n',
      'tika': '\n\n\n\n\n\n\n\n\n\n\n\n\n\

We can download this document and try it out:

In [30]:
####################################
###### Define helper functions #####
####################################
def slugify(value):
    return ''.join([c for c in value if c.isalpha() or c.isdigit() or c ==' ' or c == '.']).rstrip()

def get_filename(cd, url, title):
    if not cd:
        return f"{os.path.basename(url)[:225].strip()}{'.html' if '.' not in os.path.basename(url)[:225] else ''}" or f'{title[:225]}.html'
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return f"{os.path.basename(url)[:225].strip()}{'.html' if '.' not in os.path.basename(url)[:225] else ''}" or f'{title[:225]}.html'
    return f"{fname[0].strip()}{'.html' if '.' not in fname[0] else ''}"
####################################
####################################
####################################


# Download file
doc_name = 'KEY IPC FINDINGS : SEPTEMBER 2017 – MARCH 2018'
r = requests.get(url_path, verify=False, stream=True, allow_redirects=True)
r.raw.decode_content = True
filename = f"{slugify(get_filename(r.headers.get('content-disposition'), url_path, doc_name))}"
open(filename, 'wb').write(r.content)

/anaconda3/envs/p37/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2195054

In [38]:
tika_data = parser.from_file('/Users/brandon/repos/Document-Schema/SouthSudanKeyMessagesSept2017.pdf')

In [40]:
tika_data['content']

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1 \n \n\n\uf0b7 \n\n \n\n\uf0b7 \n\n\uf0b7 \n\n\uf0b7 \n5 \n\n\uf0b7 \n\n\uf0b7 \n\n                                                           \n\n3  \n\n \n\n\n\n2 \n \n\n\n\n3 \n \n\n\n\n4 \n \n\n\n\n5 \n \n\n\n'

## Improved `file_type` detection
Detecting the correct `file_type` is important since it allows us to determine the correct extractor (HTML vs. PDF).

Right now we use a pretty naive approach which is brittle.

In [47]:
query = {
    "aggs" : {
        "file_types" : {
            "terms" : { "field" : "file_type.keyword" } 
        }
    }
}

file_types = es.search(index=index, body=query)['aggregations']['file_types']['buckets']

print(f"There are {len(file_types)} types of files in our index. These are:")
for f in file_types:
    print(f"{f['key']}: {f['doc_count']}")

There are 10 types of files in our index. These are:
.pdf: 972
.html: 124
.pdfua1: 65
.1547047485: 9
.pdfplatformhootsuite: 3
.php: 3
: 2
.1557326170: 1
.htm: 1
.htmbm08: 1


> Note that this is problematic because unless the `file_type` is either `.html` or `.pdf` we do not parse it.

We can take a look at an example:

In [68]:
url_path = 'https://www.who.int/hac/crises/ssd/south-sudan-idsr-26march-1apr2018.pdf?ua=1'

query = {
        "query": {
            "match" : {
                "file_type.keyword" : '.pdfua1'
            }
        }
    }            
            
res = es.search(index=index, body=query)['hits']['hits']

Note that `extracted_text` is empty here, since no extraction was performed:

In [70]:
res[0]

{'_index': 'wm-dev',
 '_type': 'wm-document',
 '_id': 'e8716caf92dfcaacf0d979e402d6767804e9de9685dd0c3b46323c09d2c7b14a',
 '_score': 2.9306998,
 '_source': {'stored_url': 'https://world-modelers.s3.amazonaws.com/documents/migration/tmp/DEV/southsudanidsrannex8October2017.pdfua1',
  'file_name': 'southsudanidsrannex8October2017.pdfua1',
  'file_type': '.pdfua1',
  'category': 'November 2019 SSudan Docs',
  'extracted_text': {},
  'source_url': 'https://www.who.int/hac/crises/ssd/south-sudan-idsr-annex-8October2017.pdf?ua=1'}}

We can extract a better `file_type` with a regex:

In [71]:
source_url = res[0]['_source']['source_url']

In [72]:
import re

pattern = re.compile(r".*(\.pdf|\.html|\.docx|\.doc|\.ppt|\.pptx).*")
result = pattern.match(source_url)
result.group(1)

'.pdf'

However, this doesn't explain all the failed extractions. We can check the types of extractions and their counts:

In [73]:
query = {
    "query": {
        "exists": {
            "field": "extracted_text.bs4"
        }
    }
}

bs4_count = es.count(index=index, body=query)['count']

query = {
    "query": {
        "exists": {
            "field": "extracted_text.tika"
        }
    }
}

tika_count = es.count(index=index, body=query)['count']

query = {
    "query": {
        "exists": {
            "field": "extracted_text.pypdf2"
        }
    }
}

pypdf2_count = es.count(index=index, body=query)['count']

In [81]:
print("We extracted the following counts per extractor\n"\
      f"BS4: {bs4_count}\n"\
      f"Tika: {tika_count}\n"\
      f"PyPDF2: {pypdf2_count}"\
     )

We extracted the following counts per extractor
BS4: 56
Tika: 971
PyPDF2: 955


If there were 124 `.html` files what happened? We can check for documents with the `.html` `file_type` but that are missing `bs4` extractions:

In [99]:
query = {
    "query": {
        "query_string" : {
            "query" : "(file_type: .html) AND (NOT _exists_:extracted_text.bs4)"
        }
    }
}

res = es.search(index=index, body=query)['hits']['hits']

In [101]:
res[0]

{'_index': 'wm-dev',
 '_type': 'wm-document',
 '_id': 'e4cec19518356b6f5c107c192a779f1205622fef2d4315cf08caf25dc5552090',
 '_score': 3.8596,
 '_source': {'stored_url': 'https://world-modelers.s3.amazonaws.com/documents/migration/tmp/DEV/52304.html',
  'file_name': '52304.html',
  'file_type': '.html',
  'category': 'November 2019 SSudan Docs',
  'extracted_text': {},
  'source_url': 'https://data2.unhcr.org/en/documents/download/52304'}}

Note if we actually take a look at the `source_url` we will see that it is really a **`PDF`**, _not_ an **`HTML`** file!

In [102]:
url_path = res[0]['_source']['source_url']

In [103]:
# Download file
doc_name = 'South Sudan Situation Regional Situation'
r = requests.get(url_path, verify=False, stream=True, allow_redirects=True)
r.raw.decode_content = True
filename = f"{slugify(get_filename(r.headers.get('content-disposition'), url_path, doc_name))}"
open(filename, 'wb').write(r.content)

411942

Using Python's (magic library)[https://github.com/ahupp/python-magic] we can deal with this. 

```
pip install python-magic-bin==0.4.14
```

Note that even though the filename has `.html` it still correctly detects it as a PDF.:

In [116]:
filename

'52304.html'

In [118]:
import magic
mime = magic.Magic(mime=True)
mime.from_file(filename)

'application/pdf'

This appears to be a pretty common problem for documents from `November 2019 SSudan Docs` tab of the spreadsheet.